In [ ]:
import cv2
from tkinter import *
import tkinter.ttk as ttk
from PIL import ImageTk, Image
from tkinter import filedialog
from PIL import Image
import numpy as np
import cv2
from scipy.spatial import distance as dist
from matplotlib import pyplot as plt
import math
from PIL import Image
from PIL import ImageTk
real_real =1
ori_img = 1
Xva = 800
Yva = 480
vertex = []
src = []  # 명함 영역 꼭지점의 좌표
def wait():
    wait = cv2.waitKey(0)
    while (wait != 32):
        wait = cv2.waitKey(0)
        print(wait)


def mouse_handler(event, x, y, flags, param):  # 마우스로 좌표 알아내기
    if event == cv2.EVENT_LBUTTONUP:
        clicked = [x, y]
        print(clicked)

        # card2 - 배경과 명함의 구분이 모호한 경우

# 에지 검출 : 흑백 -> 가우시안블러링 -> 캐니
def edge_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, gray = cv2.threshold(gray, 120, 255, cv2.THRESH_TOZERO)
#     gray = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(gray, 75, 200, True)
        
    #wait()
    return edged


def contours(edge):
    global checkpnt, vertex
    # edged = edge_detection(resize_img)
    (cnts, _) = cv2.findContours(edge.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)  # 계층관계가 필요없기 때문에 contour만 추출
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]  # contourArea : contour가 그린 면적
    size= 0 
    for i in cnts:
        peri = cv2.arcLength(i, True)  # contour가 그리는 길이 반환
        approx = cv2.approxPolyDP(i, 0.02 * peri, True)  # 길이에 2% 정도 오차를 둔다

        if len(approx)>4:
            approx = approx[0:4]
        if len(approx) == 4:  # 도형을 근사해서 외곽의 꼭짓점이 4개라면 명암의 외곽으로 설정
            screenCnt = approx
            size = len(screenCnt)
            break
        if len(approx) != 4 and checkpnt == 0:  # 사각형이 그려지지 않는다면 grab_cut 실행
            pass

        if len(approx) != 4 and checkpnt > 0:
            pass
            
    if (size > 0):
        # 2.A 원래 영상에 추출한 4 변을 각각 다른 색 선분으로 표시한다.
        #cv2.line(ori_img, tuple(screenCnt[0][0]), tuple(screenCnt[size - 1][0]), (255, 0, 0), 3)
        for j in range(size - 1):
            color = list(np.random.random(size=3) * 255)
            #cv2.line(ori_img, tuple(screenCnt[j][0]), tuple(screenCnt[j + 1][0]), color, 3)
        axis = np.zeros(4)
        axis[3] = (screenCnt[3][0][1] - screenCnt[0][0][1]) / (screenCnt[3][0][0] - screenCnt[0][0][0])

        for k in range(3):
            axis[k] = (screenCnt[k][0][1] - screenCnt[k + 1][0][1]) / (screenCnt[k][0][0] - screenCnt[k + 1][0][0])
        left_axis = axis[0]  # 좌 기울기
        down_axis = axis[1]  # 하 기울기
        right_axis = axis[2]  # 우 기울기
        upper_axis = axis[3]  # 상 기울기
        print("(2.B) 순서대로 좌, 우, 상, 하 선분의 기울기")
        print(left_axis, right_axis, upper_axis, down_axis)
        print("\n")
        # 좌 선분의 y절편
        left_y = screenCnt[1][0][1] - (left_axis * screenCnt[1][0][0])  # 좌 y절편
        # 우 선분의 y절편
        right_y = screenCnt[3][0][1] - (right_axis * screenCnt[3][0][0])  # 우 y절편
        # 상 선분의 y절편
        upper_y = screenCnt[0][0][1] - (upper_axis * screenCnt[0][0][0])  # 상 y절편
        # 하 선분의 y절편
        down_y = screenCnt[2][0][1] - (down_axis * screenCnt[2][0][0])  # 하 y절편
        print("(2.B) 순서대로 좌, 우, 상, 하 선분의 y절편")
        print(left_y, right_y, upper_y, down_y)
        print("\n")
        # 양끝점의 좌표
        print("(2.B) 순서대로 좌, 우, 상, 하 선분의 양 끝점")
        print((screenCnt[0][0][0], screenCnt[0][0][1]), (screenCnt[1][0][0], screenCnt[1][0][1]))  # 좌 선분의 양 끝점
        print((screenCnt[2][0][0], screenCnt[2][0][1]), (screenCnt[3][0][0], screenCnt[3][0][1]))  # 우 성분의 양 끝점
        print((screenCnt[0][0][0], screenCnt[0][0][1]), (screenCnt[3][0][0], screenCnt[3][0][1]))  # 상 성분의 양 끝점
        print((screenCnt[1][0][0], screenCnt[1][0][1]), (screenCnt[2][0][0], screenCnt[2][0][1]))  # 하 성분의 양 끝점
        print("\n")
        # 3.B 네 꼭짓점을 각각 다른 색 점으로 표시한다. (파일이름, ?,
        #cv2.drawContours(ori_img, screenCnt, 0, (0, 0, 0), 14)  # 검
        #cv2.drawContours(ori_img, screenCnt, 1, (255, 0, 0), 14)  # 파
        #cv2.drawContours(ori_img, screenCnt, 2, (0, 255, 0), 14)  # 녹
        #cv2.drawContours(ori_img, screenCnt, 3, (0, 0, 255), 14)  # 적
        #cv2.imshow("With_Color_Image", ori_img)
        # 3.C 네 꼭지점(좌상, 좌하, 우상, 우하)의 좌표를 출력한다.
        vertex = solving_vertex(screenCnt.reshape(4, 2))
        # (topLeft, bottomLeft, topRight, bottomRight) = vertex
        print("(3.C) 순서대로 좌상, 좌하, 우상, 우하의 꼭짓점 좌표")
        print(vertex)
        vertex_switch= [vertex[0],vertex[2],vertex[3],vertex[1]]

        pts1 =  np.array(vertex_switch)

        # pts1 : 네 꼭지점의 좌표 리스트
        # pts1=np.float32([topLeft, topRight, bottomRight, bottomLeft]
        # pts1 = np.float32([[vertex[0], vertex[2], vertex[3], vertex[1]])

        w1 = abs(int(vertex[3][0]) - int(vertex[1][0])) # 아래 쪽 너비
        w2 = abs(int(vertex[2][0]) - int(vertex[0][0])) # 위 쪽 너비
        h1 = abs(int(vertex[2][1]) - int(vertex[3][1])) # 오른쪽 높이
        h2 = abs(int(vertex[0][1]) - int(vertex[1][1])) # 왼쪽 높이

        print("w1, w2, h1, h2 :",w1, w2, h1,h2)
        width = max([w1, w2])
        height = max([h1, h2])
        print("width, height",width,height)

        pts2 = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])
        print ("pts1 :",pts1,"\n","pts2 :",pts2)
        #mtrx = cv2.getPerspectiveTransform(pts1, pts2)
        #result = cv2.warpPerspective(resize_img, mtrx, (width, height))

def order_dots(pts):
    p = np.array(pts)
    xSorted = p[np.argsort(p[:, 0]), :]
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (a, c) = leftMost
    D = dist.cdist(a[np.newaxis], rightMost, "euclidean")[0]
    (b, d) = rightMost[np.argsort(D), :]
    return a, b, c, d

def solving_vertex(pts):
    global src
    points = np.zeros((4, 2), dtype="uint32")  # x,y쌍이 4개 쌍이기 때문
    # 원점 (0,0)은 맨 왼쪽 상단에 있으므로, x+y의 값이 제일 작으면 좌상의 꼭짓점 / x+y의 값이 제일 크면 우하의 꼭짓점
    s = pts.sum(axis=1)
    points[0] = pts[np.argmin(s)]  # 좌상
    points[3] = pts[np.argmax(s)]  # 우하
    # 원점 (0,0)은 맨 왼쪽 상단에 있으므로, y-x의 값이 가장 작으면 우상의 꼭짓점 / y-x의 값이 가장 크면 좌하의 꼭짓점
    diff = np.diff(pts, axis=1)
    points[2] = pts[np.argmin(diff)]  # 우상
    points[1] = pts[np.argmax(diff)]  # 좌하
    src.append(points[0])
    src.append(points[1])
    src.append(points[2])
    src.append(points[3])
    return points


def transformation():
    global width, height, src
    src_np = np.array(src, dtype=np.float32)
    # print(src_np)
    #print("zegopvertex111111", int(vertex[0][0]) -int(vertex[1][0]))
    # dst = np.float32([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]])
    global Xva, Yva
    #Xva는 가로(너비)값 : 좌측 아래점과 우측 아래점의 사이의 길이 값 대입
    Xva = math.dist(vertex[0], vertex[2])
    
    print(Xva)
    
    w1 = math.dist(vertex[0],vertex[2])
    w2 = math.dist(vertex[1],vertex[3])
    h1 = math.dist(vertex[0],vertex[1])
    h2 = math.dist(vertex[2],vertex[3])

    print("w1, w2, h1, h2 :",w1, w2, h1,h2)
    width = int(max([w1, w2]))
    height = int(max([h1, h2]))
    
    pts1 = np.float32([vertex[0],vertex[2],vertex[1],vertex[3]])
    pts2 = np.float32([[0,0],[width-1,0],[0,height-1],[width-1,height-1]])
    
    M = cv2.getPerspectiveTransform(pts1,pts2)
    
    img_result = cv2.warpPerspective(ori_img, M, (width,height))
    
    
    return img_result
    
def crack_count(last_image):
    global crack_c, real_real
    kernel = np.ones((5, 5), np.uint8)
    gray_last = cv2.cvtColor(last_image,cv2.COLOR_BGR2GRAY)#그레이스케일
    ret, binary = cv2.threshold(gray_last,100,255,cv2.THRESH_BINARY)
    #cv2.imshow("binary",binary)
    count =0
    for i in range(binary.shape[1]):
        for j in range(binary.shape[0]):
            if(binary[j,i]>90):
                count +=1
    if((width*height)!=0):
        crack_c= count*60/(width*height)
    else :
        crack_c= count*60/((width*height)+100)
    print("참고 참고 :",crack_c)
    text =grade(crack_c)
    org=(10,50)
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(last_image,text,org,font,1,(0,255,255),2)
    #cv2.imshow("finish",last_image)
    #cv2.imwrite("result.jpg", last_image)
    real_real = cv2.cvtColor(last_image, cv2.COLOR_BGR2RGB)
    real_real = Image.fromarray(real_real)
    real_real = ImageTk.PhotoImage(image=real_real)
    
def grade(crack_c) :
    if crack_c < 0.02 :
        return "A"
    elif crack_c <1:
        return "B"
    elif crack_c <2:
        return "C"
    elif crack_c <3:
        return "D"
    else:
        return "F"
    
class Point:
    def __init__(self, x, y):
        self.x = x0
        self.y = y

    def __str__(self):
        return f"[{self.x} {self.y}]"


def sfsfsfsfs(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 임계값 조절
    mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

# mask
    mask = 255 - mask

# morphology 적용
# borderconstant 사용
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# anti-alias the mask
# blur alpha channel
    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

# linear stretch so that 127.5 goes to 0, but 255 stays 255
    mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

# put mask into alpha channel
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    return result
#if __name__ == "__main__":
 #   filename = 'test1.jpeg'
  #  ori_img = cv2.imread(filename)
def mamain(ori)   :
    global ori_img
    ori_img = ori
   # ori_img = sfsfsfsfs(ori_img)
   # cv2.imshow('translate', ori_img)
  #  ori_img = cv2.resize(ori_img, dsize=(640, 480), interpolation=cv2.INTER_CUBIC)
    #resize_img = cv2.resize(ori_img, dsize=(Xva, Yva), interpolation=cv2.INTER_AREA)  # 원래 사진이 4000x3000 이라서 사이즈 조절하였음.
    global checkpnt, vertex
    global new_contour
    checkpnt = 0

    # edge_detection(grab_cut())
    h, w = ori_img.shape[:2]
    standard =20
#     for i in range(h):
#         for j in range(w):
#             if(not (ori_img[i,j][1]+standard>ori_img[i,j][2]>ori_img[i,j][1]-standard and ori_img[i,j][1]+standard > ori_img[i,j][0]>ori_img[i,j][1]-standard and ori_img[i,j][2]+standard>ori_img[i,j][1]>ori_img[i,j][2]-standard)):
#                 ori_img[i,j] = [255,255,255]
#     array = np.array([[ 1, 1], [1, 1]])
#     erosion = cv2.dilate(ori_img, array)
    edged = edge_detection(ori_img)



    #cv2.imshow("edge", edged)

    if (contours(edged)!=7) :
        print("checkpnt : ", checkpnt)
        print("\n\n\n\n\n")
        result=transformation()
        crack_count(result)
    else :
        print("사진좀 똑바로 찍으련?")

    
my_image=1

def open():
    global my_image # 함수에서 이미지를 기억하도록 전역변수 선언 (안하면 사진이 안보임)
    
    
    org_label.filename = filedialog.askopenfilename(initialdir='', title='파일선택', filetypes=(
    ('png files', '*.png'), ('jpg files', '*.jpg'), ('all files', '*.*')))
 
    print(org_label.filename) # 파일경로 view
    #Label.config(org_label)
    imim = cv2.imread(org_label.filename)
    my_image = ImageTk.PhotoImage(Image.open(org_label.filename))
    #Label(image=my_image).pack() #사진 view
    org_label.config(image = my_image)
    org_label.pack(expand="True")
    print("호랑이")
    mamain(imim)
    print("사자")
    aftr_label.config(image = real_real)
    aftr_label.pack(expand="True")
    print("고양이")
    
    # 이미지 출력
    
    
    

# 두번째 윈도우 # Tk(): 윈도우를 만드는 함수 
home = Tk()

# 윈도우 크기
home.geometry("1280x720")

# 윈도우 내용
home.title("크랙 검출하기 - 액정 켜짐 상태")

# 폰트 
#home.option_add()



#spl = ttk.Separator(home, orient = "vertical")
#spl.place(relx=0.50, rely=0, relwidth=0.5, relheight=1)




# 버튼 생성
crack_window_btn = Button(home)

# 버튼 배치
crack_window_btn.pack(side = "bottom", fill="both", expand=False)

# 버튼 크기
crack_window_btn.config(font = (20), width = 100, height = 5)

# 버튼 내용
crack_window_btn.config(text='사진 불러오기')

# 버튼 기능
crack_window_btn.config(command=open)





org_frame = ttk.Frame(home, relief="flat")
org_frame.pack(side = "left", fill="both", expand=True)


org_label = Label(org_frame) #, text ="입력사진", font = (20), width = 10, height = 0
org_label.pack(padx = 1, pady = 1, anchor = "center")





aftr_frame = ttk.Frame(home, relief="flat")
aftr_frame.pack(side="right", fill="both", expand=True)


aftr_label = Label(aftr_frame) #, text ="출력사진", font = (20), width = 10, height = 0
aftr_label.pack(padx = 10, pady = 10, anchor = "center")






home.mainloop()

/home/piai/Untitled Folder/test5.jpeg
호랑이
(2.B) 순서대로 좌, 우, 상, 하 선분의 기울기
-30.214285714285715 -84.2 0.06806282722513089 0.055


(2.B) 순서대로 좌, 우, 상, 하 선분의 y절편
2285.0714285714284 22429.2 13.895287958115183 438.645


(2.B) 순서대로 좌, 우, 상, 하 선분의 양 끝점
(75, 19) (61, 442)
(261, 453) (266, 32)
(75, 19) (266, 32)
(61, 442) (261, 453)


(3.C) 순서대로 좌상, 좌하, 우상, 우하의 꼭짓점 좌표
[[ 75  19]
 [ 61 442]
 [266  32]
 [261 453]]
w1, w2, h1, h2 : 200 191 421 423
width, height 200 423
pts1 : [[ 75  19]
 [266  32]
 [261 453]
 [ 61 442]] 
 pts2 : [[  0.   0.]
 [199.   0.]
 [199. 422.]
 [  0. 422.]]
checkpnt :  0






191.44189719076647
w1, w2, h1, h2 : 191.44189719076647 200.30227157973025 423.2316150761897 421.02969016448236
참고 참고 : 17.363120567375887
사자
고양이
